# Description

TODO

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
from pathlib import Path

import numpy as np
import pandas as pd
from IPython.display import HTML
from tqdm import tqdm

from entity import Trait, Gene

import conf

# Settings

In [3]:
EXPERIMENT_NAME = "lv"
LIPIDS_GENE_SET = "gene_set_decrease"

In [4]:
RANDOM_SEED = 0
N_PERMUTATIONS = 1000
N_TOP_TRAITS = 25

In [5]:
OUTPUT_DIR = Path(
    conf.RESULTS["CRISPR_ANALYSES"]["BASE_DIR"],
    f"{EXPERIMENT_NAME}-{LIPIDS_GENE_SET}",
    "permutations",
)
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)
display(OUTPUT_DIR)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/crispr_analyses/lv-gene_set_decrease/permutations')

# Data loading

## Traits list

In [6]:
input_filepath = OUTPUT_DIR / "lipid_traits_list.pkl"
display(input_filepath)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/crispr_analyses/lv-gene_set_decrease/permutations/lipid_traits_list.pkl')

In [7]:
with open(input_filepath, "rb") as handle:
    lipids_related_traits = pickle.load(handle)

In [8]:
lipids_related_traits

{'4079_raw-Diastolic_blood_pressure_automated_reading',
 '4080_raw-Systolic_blood_pressure_automated_reading',
 '6150_100-Vascularheart_problems_diagnosed_by_doctor_None_of_the_above',
 '6150_4-Vascularheart_problems_diagnosed_by_doctor_High_blood_pressure',
 'I9_UAP-Unstable_angina_pectoris',
 'K11_COELIAC-Coeliac_disease',
 'atherosclerosis',
 'celiac disease',
 'hypertension',
 'malabsorption syndrome'}

## Permutations

In [9]:
input_filepath = OUTPUT_DIR / "permutation_results.pkl"
display(input_filepath)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/crispr_analyses/lv-gene_set_decrease/permutations/permutation_results.pkl')

In [10]:
with open(input_filepath, "rb") as handle:
    permutation_results = pickle.load(handle)

In [11]:
df = pd.Series([x for l in permutation_results for x in l])

In [12]:
with pd.option_context(
    "display.max_rows", None, "display.max_columns", None, "max_colwidth", None
):
    display(df.value_counts())

celiac disease                                                           225
K11_COELIAC-Coeliac_disease                                              126
malabsorption syndrome                                                   121
6150_100-Vascularheart_problems_diagnosed_by_doctor_None_of_the_above     19
4080_raw-Systolic_blood_pressure_automated_reading                        19
6150_4-Vascularheart_problems_diagnosed_by_doctor_High_blood_pressure     18
atherosclerosis                                                           15
4079_raw-Diastolic_blood_pressure_automated_reading                       14
hypertension                                                              13
I9_UAP-Unstable_angina_pectoris                                            9
dtype: int64

# p-value

In [13]:
# in this case we are permisive to compute the p-value, and count cases where at least half of the important traits are among the top
pval = (
    sum(
        [
            (i / len(lipids_related_traits)) > 0.50
            for i in list(map(len, permutation_results))
        ]
    )
    + 1
) / (len(permutation_results) + 1)

display(pval)

0.001998001998001998

In [14]:
assert pval < 0.05